# Agente LLM Personalizado (con un modelo de Chat y Memoria)
---

Este cuaderno explica cómo crear su propio agente personalizado basado en un modelo de chat. 

Un agente de chat LLM consta de tres partes:

- `PromptTemplate`: Esta es la plantilla de indicaciones que se puede utilizar para indicarle al modelo de lenguaje qué hacer
- `ChatModel`: Este es el modelo de lenguaje que impulsa al agente.
- `stop sequence`: Le indica al LLM que deje de generar tan pronto como se encuentre esta cadena
- `OutputParser`: Esto determina cómo analizar la salida de `LLM` en un objeto `AgentAction` o `AgentFinish`.

El Agente `LLM` se utiliza en un `AgentExecutor`. Este `AgentExecutor` puede considerarse en gran medida como un bucle que:

1. Pasa la entrada del usuario y cualquier paso previo al Agente (en este caso, el Agente LLM)
2. Si el Agente devuelve un `AgentFinish`, devuélvalo directamente al usuario.
3. Si el Agente devuelve una `AgentAction`, úsela para llamar a una herramienta (`tool`) y obtener una Observación (`Observation`)
4. Repita, pasando `AgentAction` y `Observation` nuevamente al Agente hasta que se emita un `AgentFinish`.

`AgentAction` es una respuesta que consta de `action` y `action_input`. 

- `action` se refiere a qué herramienta usar 
- `action_input` se refiere a la entrada a esa herramienta. 
- `log` también se puede proporcionar como más contexto (que se puede usar para registrar, rastrear, etc.).

`AgentFinish` es una respuesta que contiene el mensaje final que se enviará al usuario. Esto debe usarse para finalizar la ejecución de un agente.

## Configuracion del Entorno
---

Realizar las importaciones necesarias.

In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from typing import List, Union
import re

load_dotenv(find_dotenv())

True

## Configurar herramienta (tools)
---

Configure cualquier herramienta (`tool`) que el agente quiera utilizar. 

Puede ser necesario incluir esto en el `prompt` (para que el agente sepa cómo utilizar estas herramientas).

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

## Plantilla de Prompt (Prompt Template)
---

Esto le indica al agente qué hacer. Generalmente, la plantilla debe incorporar:

- `tools`: a qué herramientas tiene acceso el agente y cómo y cuándo llamarlas.
- `intermediate_steps`: Estas son tuplas de pares anteriores (`AgentAction`, `Observation`). Por lo general, estos no se pasan directamente al modelo, pero el `prompt template` les da formato de una manera específica.
- `input`: entrada genérica del usuario.

In [3]:
# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin!

Previous conversation history:
{chat_history}

Question: {input}
{agent_scratchpad}"""

In [4]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts

        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [5]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "chat_history"]
)

## Analizador de salida (Output parser)
---

El analizador de salida (output parser) es responsable de analizar la salida de LLM en `AgentAction` y `AgentFinish`. Por lo general, esto depende en gran medida del mensaje utilizado.

Aquí es donde puede cambiar el análisis para realizar reintentos, manejar espacios en blanco, etc.

In [6]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [7]:
output_parser = CustomOutputParser()

## Configurar el LLM
---

¡Elija el LLM que desea utilizar!

Para este ejemplo usaremos ChatOpenAI

In [8]:
llm = ChatOpenAI(temperature=0)

## Definir la secuencia de parada (stop sequence)
---

Esto es importante porque le indica al `LLM` cuándo detener la generación.

Esto depende en gran medida del indicador y del modelo que esté utilizando. Generalmente, desea que sea cualquier token que use en el mensaje para indicar el inicio de una observación (de lo contrario, el LLM puede alucinar una observación para usted).

## Configurar el Agente
---

Ahora podemos combinar todo para configurar nuestro agente:

In [9]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
)

In [10]:
memory = ConversationBufferMemory(
    memory_key="chat_history"
)

In [11]:
tool_names = [tool.name for tool in tools]

In [12]:
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names,
)

## Utilice el agente
---

¡Ahora podemos usarlo!

Comencemos por crear el "ejecutor"

In [13]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
)

Ahora hagamos una pregunta a nuestro agente, por ejemplo preguntemos ¿cuantas persoas viven en canada en el año 2023?

In [14]:
agent_executor.run("How many people live in Canada as of 2023?")



> Entering new AgentExecutor chain...
Thought: I don't have the current population data for Canada in 2023.
Action: Search
Action Input: "current population of Canada 2023"

Observation:38,781,291 people
I now know the current population of Canada in 2023.
Final Answer: The current population of Canada as of 2023 is 38,781,291 people.

> Finished chain.


'The current population of Canada as of 2023 is 38,781,291 people.'

In [15]:
agent_executor.run("how about in mexico?")



> Entering new AgentExecutor chain...
Thought: I need to find the current population of Mexico.
Action: Search
Action Input: "current population of Mexico"

Observation:{'type': 'population_result', 'place': 'Mexico', 'population': '126.7 million', 'year': '2021'}
The current population of Mexico is 126.7 million as of 2021.
Final Answer: The current population of Mexico is 126.7 million.

> Finished chain.


'The current population of Mexico is 126.7 million.'

Como se puede observar el agente al no tener "estado", no comprende la pregunta en la conversacion.

Tendriamos que hacer la pregunta directamente.

In [16]:
agent_executor.run("how about colombia?")



> Entering new AgentExecutor chain...
Thought: I need to find the current population of Colombia.
Action: Search
Action Input: "current population of Colombia"

Observation:{'type': 'population_result', 'place': 'Colombia', 'population': '51.52 million', 'year': '2021'}
The current population of Colombia is 51.52 million as of 2021.
Final Answer: The current population of Colombia is 51.52 million.

> Finished chain.


'The current population of Colombia is 51.52 million.'

In [17]:
agent_executor.run("what is their national anthem called?")



> Entering new AgentExecutor chain...
Thought: I need to find out the name of the national anthem for Canada, Mexico, and Colombia.
Action: Search
Action Input: "national anthem of Canada"

Observation:['"O Canada" is the national anthem of Canada. The song was originally commissioned by Lieutenant Governor of Quebec Théodore Robitaille for the 1880 Saint-Jean-Baptiste Day ceremony; Calixa Lavallée composed the music, after which words were written by the poet and judge Sir Adolphe-Basile Routhier.', 'O Canada type: National anthem.', 'O Canada main_tab_text: Overview.', 'O Canada kgmid: /m/05mtn.', 'O Canada lyricists: Calixa Lavallée, Robert Stanley Weir, Adolphe-Basile Routhier.', 'O Canada composer: Calixa Lavallée.', 'O Canada key: G major.', 'O Canada adopted: July 1, 1980.', 'O Canada also_known_as: French - Ô Canada.', 'It was proclaimed the official national anthem on July 1, 1980. “God Save the Queen” remains the royal anthem of Canada. The music, written by ...']
I need to

'The national anthem of Canada is called "O Canada", the national anthem of Mexico is called "Mexican National Anthem", and the national anthem of Colombia is called "National Anthem of the Republic of Colombia".'